<a href="https://colab.research.google.com/github/rivera-squared/Educacion_Machine_Learning/blob/main/educacion_DeepLearning_newmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Model


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from tensorflow.keras.optimizers import SGD

In [3]:
# Import the SGD optimizer
from keras.callbacks import EarlyStopping

In [4]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [60]:
# New data set with 'cambio_matricula' column
x = pd.read_csv('https://raw.githubusercontent.com/edghero/data/main/escuelas_nuevas.csv').dropna()

In [61]:
x['cambio_matricula'] = pd.to_numeric(x['cambio_matricula'], errors = 'coerce')

In [69]:
# Setting index
x = x.set_index('escuela')

In [62]:
# renaming nivel to match DE's current naming convention. I NEED TO GO OVER INTERLOCKING ONE BY ONE TO RECLASS THEM PROPERLY
x['nivel'] = np.where(x['nivel'] == 'elemental', 'primario',
                      np.where(x['nivel'] == 'intermedia','primario',
                               np.where(x['nivel'] == 'especializada','todos los niveles',
                                        np.where(x['nivel'] == 'superior', 'secundario',
                                                 np.where(x['nivel'] == 'interlocking','todos los niveles','otro')))))

In [72]:
primario = x[x['nivel'] == 'primario'].drop(columns = ['nivel','escuela_receptora','matricula_2016-17','ANO_ESCOLAR','CODIGO_ESCUELA','municipio'])
primario = pd.get_dummies(primario).drop(columns = 'region_SAN JUAN')
primario

,consolidada,matricula,cambio_matricula,promedio_espanol,promedio_matematica,promedio_ingles,promedio_ciencias,region_ARECIBO,region_BAYAMON,region_CAGUAS,region_HUMACAO,region_MAYAGUEZ,region_PONCE
escuela,,,,,,,,,,,,,
Agapito Rosario Rosario,0,473,-0.11,57.8,65.3,67.3,77.1,1,0,0,0,0,0
Amalia Lopez de Avila (Nueva),0,332,1.24,70.4,76.7,76.1,90.4,1,0,0,0,0,0
Angel Sandin Martinez,0,473,-0.11,57.8,65.3,67.3,77.1,1,0,0,0,0,0
Antonio Paoli,0,294,0.15,36.6,53.2,20.9,44.1,1,0,0,0,0,0
Antonio Reyes,0,428,-0.04,42.6,18.7,45.8,60.7,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jesus Silva,0,309,-0.04,73.0,83.4,43.4,67.4,0,0,0,0,0,0
Jose F Diaz,0,229,-0.39,24.0,43.8,17.9,40.0,0,0,0,0,0,0
Villa Capri,0,222,-0.13,96.8,98.9,92.6,100.0,0,0,0,0,0,0


In [73]:
primario.head(2)

,consolidada,matricula,cambio_matricula,promedio_espanol,promedio_matematica,promedio_ingles,promedio_ciencias,region_ARECIBO,region_BAYAMON,region_CAGUAS,region_HUMACAO,region_MAYAGUEZ,region_PONCE
escuela,,,,,,,,,,,,,
Agapito Rosario Rosario,0,473,-0.11,57.8,65.3,67.3,77.1,1,0,0,0,0,0
Amalia Lopez de Avila (Nueva),0,332,1.24,70.4,76.7,76.1,90.4,1,0,0,0,0,0


In [74]:
X = primario.drop(columns = 'consolidada')
y = primario['consolidada']

In [75]:
predictors_primario = X
target_primario = to_categorical(y)

First Deep Learning Model for 'escuelas primarias' with 'Adam' optimizer

In [78]:
early_stopping_monitor = EarlyStopping(patience=5)

In [79]:
# Set up the model: model
n_cols = predictors_primario.shape[1]
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=(n_cols,))) # Add the first layer
model.add(Dense(30, activation="relu")) # Add the second layer
model.add(Dense(50, activation="sigmoid")) # Add the second layer
model.add(Dense(2, activation = 'softmax')) # Add the output layer; needed to add softmax activation since this method is for classifiication problem set

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ['accuracy'])

model.fit(predictors_primario, target_primario, validation_split = 0.3, epochs = 50, callbacks = [early_stopping_monitor])

Epoch 1/50
15/15 [==============================] - 1s 18ms/step - loss: 0.7230 - accuracy: 0.6346 - val_loss: 0.5287 - val_accuracy: 0.7449
Epoch 2/50
15/15 [==============================] - 0s 5ms/step - loss: 0.5483 - accuracy: 0.7243 - val_loss: 0.5140 - val_accuracy: 0.7653
Epoch 3/50
15/15 [==============================] - 0s 5ms/step - loss: 0.5248 - accuracy: 0.7396 - val_loss: 0.5083 - val_accuracy: 0.7602
Epoch 4/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5225 - accuracy: 0.7462 - val_loss: 0.5052 - val_accuracy: 0.7653
Epoch 5/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5145 - accuracy: 0.7549 - val_loss: 0.5109 - val_accuracy: 0.7551
Epoch 6/50
15/15 [==============================] - 0s 4ms/step - loss: 0.5120 - accuracy: 0.7505 - val_loss: 0.5020 - val_accuracy: 0.7602
Epoch 7/50
15/15 [==============================] - 0s 5ms/step - loss: 0.5126 - accuracy: 0.7484 - val_loss: 0.4990 - val_accuracy: 0.7704
Epoch 8/50
15/15 [=

In [80]:
# Calculate predictions
predictions_primario = model.predict(predictors_primario)

In [81]:
primario['predictions'] = predictions_primario[:,1]

In [82]:
primario['predictions'] = round(primario.predictions)

In [83]:
print(confusion_matrix(primario['consolidada'], primario['predictions']))
print(classification_report(primario['consolidada'], primario['predictions']))

[[377  66]
 [ 90 120]]
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       443
           1       0.65      0.57      0.61       210

    accuracy                           0.76       653
   macro avg       0.73      0.71      0.72       653
weighted avg       0.76      0.76      0.76       653



In [ ]:
from keras.models import load_model

In [ ]:
model.save('model_DL_adam.h5')

Deep Learning Model with SGD optimizer with different learning rates

In [ ]:
early_stopping_monitor = EarlyStopping(patience=2)

In [84]:
n_cols = predictors_primario.shape[1]

lr_to_test = [.000001, .001, .00025]

# loop over learning rates
for lr in lr_to_test:
  print('\n\nTesting model with learning rate: %f\n'%lr )
  model = Sequential()
  model.add(Dense(100, activation="relu", input_shape=(n_cols,))) # Add the first layer
  model.add(Dense(30, activation="relu")) # Add the second layer
  model.add(Dense(50, activation="relu")) # Add the second layer
  model.add(Dense(2, activation = 'softmax')) # Add the output layer; needed to add softmax activation since this method is for classifiication problem set
  my_optimizer = SGD(learning_rate=lr)
  model.compile(optimizer= my_optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
  #print("Loss function: " + model.loss)
  model.fit(predictors_primario, target_primario, validation_split=0.3, epochs= 50, callbacks=[early_stopping_monitor])




Testing model with learning rate: 0.000001

Epoch 1/50
15/15 [==============================] - 1s 16ms/step - loss: 36.7742 - accuracy: 0.3414 - val_loss: 39.4938 - val_accuracy: 0.2755
Epoch 2/50
15/15 [==============================] - 0s 4ms/step - loss: 35.9327 - accuracy: 0.3414 - val_loss: 38.5683 - val_accuracy: 0.2755
Epoch 3/50
15/15 [==============================] - 0s 5ms/step - loss: 35.0976 - accuracy: 0.3414 - val_loss: 37.6550 - val_accuracy: 0.2755
Epoch 4/50
15/15 [==============================] - 0s 5ms/step - loss: 34.2752 - accuracy: 0.3414 - val_loss: 36.7582 - val_accuracy: 0.2755
Epoch 5/50
15/15 [==============================] - 0s 5ms/step - loss: 33.4692 - accuracy: 0.3414 - val_loss: 35.8773 - val_accuracy: 0.2755
Epoch 6/50
15/15 [==============================] - 0s 5ms/step - loss: 32.6762 - accuracy: 0.3414 - val_loss: 35.0158 - val_accuracy: 0.2755
Epoch 7/50
15/15 [==============================] - 0s 5ms/step - loss: 31.8977 - accuracy: 0.3414 - 

####Model with SGD as optimizer and learning rate of 0.00025

In [85]:
early_stopping_monitor = EarlyStopping(patience=5)

In [86]:
n_cols = predictors_primario.shape[1]

lr_to_test = [.00025]

# loop over learning rates
for lr in lr_to_test:
  print('\n\nTesting model with learning rate: %f\n'%lr )
  model = Sequential()
  model.add(Dense(100, activation="relu", input_shape=(n_cols,))) # Add the first layer
  model.add(Dense(30, activation="relu")) # Add the second layer
  model.add(Dense(50, activation="relu")) # Add the second layer
  model.add(Dense(2, activation = 'softmax')) # Add the output layer; needed to add softmax activation since this method is for classifiication problem set
  my_optimizer = SGD(learning_rate=lr)
  model.compile(optimizer= my_optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
  #print("Loss function: " + model.loss)
  model.fit(predictors_primario, target_primario, validation_split=0.3, epochs= 50, callbacks=[early_stopping_monitor])




Testing model with learning rate: 0.000250

Epoch 1/50
15/15 [==============================] - 1s 23ms/step - loss: 3.0333 - accuracy: 0.5646 - val_loss: 0.7375 - val_accuracy: 0.6480
Epoch 2/50
15/15 [==============================] - 0s 5ms/step - loss: 0.7316 - accuracy: 0.7112 - val_loss: 0.6417 - val_accuracy: 0.6633
Epoch 3/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6747 - accuracy: 0.7112 - val_loss: 4.9254 - val_accuracy: 0.2704
Epoch 4/50
15/15 [==============================] - 0s 5ms/step - loss: 0.8612 - accuracy: 0.6871 - val_loss: 0.5681 - val_accuracy: 0.6939
Epoch 5/50
15/15 [==============================] - 0s 5ms/step - loss: 0.6343 - accuracy: 0.6915 - val_loss: 0.5630 - val_accuracy: 0.6990
Epoch 6/50
15/15 [==============================] - 0s 4ms/step - loss: 0.8764 - accuracy: 0.6630 - val_loss: 0.9041 - val_accuracy: 0.7449
Epoch 7/50
15/15 [==============================] - 0s 5ms/step - loss: 0.8963 - accuracy: 0.6871 - val_loss: 0.6

In [87]:
# Calculate predictions
predictions_primario = model.predict(predictors_primario)

In [88]:
primario['predictions'] = predictions_primario[:,1]

In [89]:
primario['predictions'] = round(primario.predictions)

In [ ]:
primario

In [90]:
print(confusion_matrix(primario['consolidada'], primario['predictions']))
print(classification_report(primario['consolidada'], primario['predictions']))

[[425  18]
 [164  46]]
              precision    recall  f1-score   support

           0       0.72      0.96      0.82       443
           1       0.72      0.22      0.34       210

    accuracy                           0.72       653
   macro avg       0.72      0.59      0.58       653
weighted avg       0.72      0.72      0.67       653

